# Exact Diagonalization of a 1D Spin Chain
<b> Christina Lee

Category: Grad

Prerequisites: Many-Body Quantum Mechanics </b>

# Table of Contents
 <p><div class="lev1"><a href="#Exact-Diagonalization-of-a-1D-Spin-Chain">Exact Diagonalization of a 1D Spin Chain</a></div><div class="lev2"><a href="#Theoretical-Background">Theoretical Background</a></div><div class="lev2"><a href="#Initializing-Variables">Initializing Variables</a></div><div class="lev2"><a href="#Creating-Basis-Vectors">Creating Basis Vectors</a></div><div class="lev3"><a href="#Conservation-of-Magnetization">Conservation of Magnetization</a></div><div class="lev5"><a href="#The-Hamiltonian-Operator">The Hamiltonian Operator</a></div><div class="lev2"><a href="#For-Further-Information">For Further Information</a></div><div class="lev1"><a href="#A-different-way-I'm-trying">A different way I'm trying</a></div>

## Theoretical Background 
\begin{equation}
\mathcal{H}= J \sum\limits_{\langle i,j \rangle} \mathbf{S}_i \cdot \mathbf{S}_j
\end{equation}
\begin{equation}
\mathcal{H} = J \sum\limits_{\langle i,j \rangle}\left[ S_i^z S_j^z + \frac{1}{2} \left(S^+_iS^-_j + S^-_i S^+_j \right)\right]
\end{equation}

\begin{equation}
\mathcal{H}|\Psi (...,i,j,...) \rangle 
    = \left( \sum\limits_{\langle i,j \rangle} S^z_i S^z_j \right) | \Psi (...,i,j,...) \rangle +
    \sum\limits_{\langle i,j \rangle} 
\end{equation}

In [1]:
# Pkg.update()
# Pkg.add("Iterators")
# Pkg.add("PyPlot")
using Iterators
using PyPlot

INFO: Recompiling stale cache file /Users/mira/.julia/lib/v0.4/Iterators.ji for module Iterators.


## Initializing Variables

In [3]:
n=4
nstates=2^n

16

## Creating Basis Vectors

If we were doing this with a different language, we could actually represent our ones and zeros with individual bits instead of the collection of 8-bits in our `Int8`. But right now lets do a trade off of computational efficiency versus coding efficiency and call this implementation sufficient.

In [35]:
Ψv=collect(product(repeated(convert(Int8,0):convert(Int8,1),n)...))

16-element Array{Tuple{Int8,Int8,Int8,Int8},1}:
 (0,0,0,0)
 (1,0,0,0)
 (0,1,0,0)
 (1,1,0,0)
 (0,0,1,0)
 (1,0,1,0)
 (0,1,1,0)
 (1,1,1,0)
 (0,0,0,1)
 (1,0,0,1)
 (0,1,0,1)
 (1,1,0,1)
 (0,0,1,1)
 (1,0,1,1)
 (0,1,1,1)
 (1,1,1,1)

In [73]:
Ψvn=Array{Array{Int8}}(nstates)
for i in 1:nstates
    Ψvn[i]=collect(Ψv[i])
end

In [74]:
Ψvn

16-element Array{Array{Int8,N},1}:
 Int8[0,0,0,0]
 Int8[1,0,0,0]
 Int8[0,1,0,0]
 Int8[0,0,1,0]
 Int8[0,0,0,1]
 Int8[1,1,0,0]
 Int8[1,0,1,0]
 Int8[0,1,1,0]
 Int8[1,0,0,1]
 Int8[0,1,0,1]
 Int8[0,0,1,1]
 Int8[1,1,1,0]
 Int8[1,1,0,1]
 Int8[1,0,1,1]
 Int8[0,1,1,1]
 Int8[1,1,1,1]

### Conservation of Magnetization

If we seperate our Hilbert Space into subspaces of conserved quantities, like total magnetization, that shrinks the size of our problem.  Here I create a vector `m` which holds the total magnetization for each of our basis states.  
    
We can then use a Julia sorting function to sort both `m` and `Ψv` into blocks of constant magnetization.

In [62]:
m=zeros(Int64,nstates)
for i in 1:nstates
    m[i]=sum(Ψvn[i]-.5)
end
ind=sortperm(m)
m=m[ind]
Ψvn=Ψvn[ind]
[Ψvn m]

16x2 Array{Any,2}:
 Int8[0,0,0,0]  -2
 Int8[1,0,0,0]  -1
 Int8[0,1,0,0]  -1
 Int8[0,0,1,0]  -1
 Int8[0,0,0,1]  -1
 Int8[1,1,0,0]   0
 Int8[1,0,1,0]   0
 Int8[0,1,1,0]   0
 Int8[1,0,0,1]   0
 Int8[0,1,0,1]   0
 Int8[0,0,1,1]   0
 Int8[1,1,1,0]   1
 Int8[1,1,0,1]   1
 Int8[1,0,1,1]   1
 Int8[0,1,1,1]   1
 Int8[1,1,1,1]   2

In [75]:
Ψvm=Array{Array{Array{Int8}}}(n+1)
lengths=zeros(Int16,n+1);
old=1
new=1
for i in 1:(n+1)
    Ψvm[i]=[]
    for j in old:new
       push!(Ψvm[i],Ψvn[j]) 
    end
    old=new+1
    new=old+binomial(n,i)-1
    lengths[i]=binomial(n,i-1)
end

In [105]:
[1:6 Ψvm[3]]

6x2 Array{Any,2}:
 1  Int8[1,1,0,0]
 2  Int8[1,0,1,0]
 3  Int8[0,1,1,0]
 4  Int8[1,0,0,1]
 5  Int8[0,1,0,1]
 6  Int8[0,0,1,1]

##### The Hamiltonian Operator

In [86]:
state=2
i=lengths[state]
M=zeros(i,i)
for j in 1:i
    M[j,j]=m[i]
end
M[1,2]=.5
M[2,1]=.5
M[2,3]=.5
M[3,2]=.5
M[4,3]=.5
M[3,4]=.5

M

4x4 Array{Float64,2}:
 -1.0   0.5   0.0   0.0
  0.5  -1.0   0.5   0.0
  0.0   0.5  -1.0   0.5
  0.0   0.0   0.5  -1.0

In [123]:
test=[0;Ψvm[3]]-[Ψvm[3];0]
test=test[2:end-1]
Ψvm[3]

6-element Array{Array{Int8,N},1}:
 Int8[1,1,0,0]
 Int8[1,0,1,0]
 Int8[0,1,1,0]
 Int8[1,0,0,1]
 Int8[0,1,0,1]
 Int8[0,0,1,1]

In [125]:
test2=[count(i->i==1,test[i]) for i in 1:5]

5-element Array{Int64,1}:
 1
 1
 2
 1
 1

In [82]:
for k in 1:n
    i=lengths[k]
    M=zeros(i,i)
    for j in 1:i
        M[j,j]=m[i]
    end
    println(M)
end

[-2.0]
[-1.0 0.0 0.0 0.0
 0.0 -1.0 0.0 0.0
 0.0 0.0 -1.0 0.0
 0.0 0.0 0.0 -1.0]
[0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0]
[-1.0 0.0 0.0 0.0
 0.0 -1.0 0.0 0.0
 0.0 0.0 -1.0 0.0
 0.0 0.0 0.0 -1.0]


In [24]:
M

1x1 Array{Float64,2}:
 0.0

In [ ]:
function GetOperator(basis,n)
	n2=2^n
	M=zeros(n2,n2)
    maxbin=length(bin(n2-1))

	for j=1:n2

        for k=1:(n2/2+1)
            count='2'
            if j!=k
                for m=1:maxbin
                    if basis[j][m]!=basis[k][m] && count=='2'
                        count=basis[j][m]
                    elseif basis[j][m]!=basis[k][m] && basis[j][m]!=count && count !='2'
                        count='3'
                        M[j,k]=.5
                        M[k,j]=.5
                    elseif basis[j][m]!=basis[k][m] && count=='3'
                        M[j,k]=0
                        M[k,j]=0
                    end
                end

            end
        end
	end
	return M
end


## For Further Information
* http://physics.bu.edu/~sandvik/vietri/dia.pdf


# A different way I'm trying

In [3]:
n=4
nstates=2^n

16

In [4]:
psi=collect(0:(nstates-1))
for p in psi
    println(bin(p,4),' ',p)
end

0000 0
0001 1
0010 2
0011 3
0100 4
0101 5
0110 6
0111 7
1000 8
1001 9
1010 10
1011 11
1100 12
1101 13
1110 14
1111 15


In [5]:
powers2=collect(0:(n-1))
powers2=2.^powers2
mask=[0;powers2]+[powers2;0]
mask=mask[2:end-1]
for i in 1:(length(mask))
    println(bin(mask[i]))
end


11
110
1100


In [48]:
m=zeros(psi)
for i in 1:nstates
    m[i]=sum((psi[i]&powers2)./(powers2))
    #println(round(Int,(psi[i]&powers2)./powers2))
end
m

16-element Array{Int64,1}:
 0
 1
 1
 1
 1
 2
 2
 2
 2
 2
 2
 3
 3
 3
 3
 4

In [50]:
ind=sortperm(m)
m=(m[ind]-n/2)/2
psi=psi[ind]
[psi m]

16x2 Array{Float64,2}:
  0.0  -1.0
  1.0  -0.5
  2.0  -0.5
  4.0  -0.5
  8.0  -0.5
  3.0   0.0
  5.0   0.0
  6.0   0.0
  9.0   0.0
 10.0   0.0
 12.0   0.0
  7.0   0.5
 11.0   0.5
 13.0   0.5
 14.0   0.5
 15.0   1.0

In [8]:
psia=Array{Array{Int64}}(n+1)
old=1
new=1
for i in 1:(n+1)
    #println(old,' ',new)
    psia[i]=psi[old:new]
    old=new+1
    new=new+binomial(n,i)
end
psia

5-element Array{Array{Int64,N},1}:
 [0]            
 [1,2,4,8]      
 [3,5,6,9,10,12]
 [7,11,13,14]   
 [15]           

In [58]:
i=3
dim=binomial(n,i-1)
M=(i-1-n/2)/2*eye(Float64,dim,dim)

6x6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

In [60]:
test=psia[i]

6-element Array{Int64,1}:
  3
  5
  6
  9
 10
 12

In [70]:
for p in test
    println(bin(p,4),' ',p)
end

0011 3
0101 5
0110 6
1001 9
1010 10
1100 12


In [68]:
for a in 1:6
    temp=round(Int,(test[a]&powers2)./powers2)
    println(temp)
    for b in 1:(n-1)
        if temp[b+1]!=temp[b]
            println(temp,' ',test[a] $ mask[b], ((test[a]$mask[b]) & powers2)./powers2)
            nonzero=test[a] $ mask[b]
            c=findc(nonzero,test)
            println(c)
            M[a,c]=.5
            M[c,a]=.5
        end
    end
end

[1

LoadError: LoadError: indexing Array{Float64,2} with types Tuple{Int64,Void} is not supported
while loading In[68], in expression starting on line 1

,1,0,0]
[1,1,0,0] 5[1.0,0.0,1.0,0.0]
2
[1,0,1,0]
[1,0,1,0] 6[0.0,1.0,1.0,0.0]
3
[1,0,1,0] 3[1.0,1.0,0.0,0.0]
nothing


In [42]:
function findc(nonzero::Int,test::Array)
    
    lp=ceil(Int,length(test)/2)
    while lp>1
        #println(lp)
        if(nonzero==test[lp] )
            return lp
        elseif nonzero < test[lp] 
            #println("less than ",nonzero,' ',test[lp],' ',lp)
            test=test[1:(lp)]
            lp=ceil(Int,length(test)/2)
            
        else
            #println("greater than ",nonzero,' ',test[lp])
            test=test[(lp):end]
            lp=ceil(Int,length(test)/2)
        end
    end
end

findc (generic function with 1 method)